In [1]:
import joblib
from scapy.all import sniff, IP, TCP, UDP, ICMP
import pandas as pd
import numpy as np


In [2]:
model = joblib.load('networkdetection.joblib')

In [3]:
feature_names = ['id.orig_p', 'id.resp_p', 'orig_ip_bytes', 'packet_size', 'proto_icmp', 
                 'proto_tcp', 'proto_udp', 'conn_state_REJ', 'conn_state_RSTO', 
                 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 
                 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_SF', 
                 'conn_state_SH', 'conn_state_SHR']

# Function to encode connection state
def encode_conn_state(conn_state):
    states = ['REJ', 'RSTO', 'RSTOS0', 'RSTR', 'RSTRH', 'S0', 'S1', 'S2', 'SF', 'SH', 'SHR']
    encoding = [True if conn_state == state else False for state in states]
    return encoding

# Function to dynamically get the connection state based on packet attributes
def get_conn_state(packet):
    if IP in packet:
        # If it's a TCP packet
        if TCP in packet:
            tcp_flags = packet[TCP].flags
            
            if tcp_flags == 0x04:  # RST
                return 'RSTO'
            elif tcp_flags == 0x14:  # RST and ACK
                return 'RSTR'
            elif tcp_flags == 0x02:  # SYN
                return 'S0'
            elif tcp_flags == 0x12:  # SYN and ACK
                return 'S1'
            elif tcp_flags == 0x10:  # ACK (for ongoing connection)
                return 'SF'
            elif tcp_flags == 0x01:  # FIN
                return 'SH'
            elif tcp_flags == 0x11:  # FIN and ACK
                return 'SHR'  # Other TCP states

        # If it's a UDP packet (no connection states in UDP, so default to ongoing state)
        elif UDP in packet:
            return 'SF'  # Ongoing flow, can customize if needed

        # If it's an ICMP packet
        elif ICMP in packet:
            return 'SF'  # Can adjust to a specific ICMP flow state if required

        # Handle other cases (e.g., rejected packets or unknown protocols)
        else:
            return 'REJ'
    return 's0'
 # Default for other packet types

# Function to extract features from packets
def extract_features_from_packet(packet):
    packet_size = len(packet)
    orig_ip_bytes = len(packet[IP]) if IP in packet else 0
    proto_icmp = True if ICMP in packet else False
    proto_tcp = True if TCP in packet else False
    proto_udp = True if UDP in packet else False
    
    # Dynamically get the connection state
    conn_state = get_conn_state(packet)
    
    # One-hot encode the connection state
    conn_state_encoded = encode_conn_state(conn_state)

    # Combine features into a single array
    features = [
        packet[IP].sport if TCP in packet or UDP in packet else 0,  # id.orig_p
        packet[IP].dport if TCP in packet or UDP in packet else 0,  # id.resp_p
        orig_ip_bytes,  # orig_ip_bytes
        packet_size,  # packet_size
        proto_icmp,  # proto_icmp
        proto_tcp,  # proto_tcp
        proto_udp  # proto_udp
    ] + conn_state_encoded

    # Ensure the length of features is 18
    if len(features) != 18:
        print(f"Feature length mismatch: {len(features)}. Features: {features}")

    return np.array(features).reshape(1, -1)

# Function to handle each packet
 # Output prediction
def packet_handler(packet):
    # Extract features from the packet
    features = extract_features_from_packet(packet)
    
    # List of feature names used when training the model
    feature_names = ['id.orig_p', 'id.resp_p', 'orig_ip_bytes', 'packet_size', 'proto_icmp', 
                     'proto_tcp', 'proto_udp', 'conn_state_REJ', 'conn_state_RSTO', 
                     'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 
                     'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_SF', 
                     'conn_state_SH', 'conn_state_SHR']

    # Convert the features to a DataFrame with proper feature names
    features_df = pd.DataFrame(features, columns=feature_names)
    
    # Get prediction from the model
    prediction = model.predict(features_df)
    
    # Output the prediction and packet summary
    if prediction[0] == 1:  # Assuming '1' represents malicious
        print(f"Packet: {packet.summary()} {prediction}| Malicious")
# Start sniffing on the desired interface

def start_sniffing():
    print("Starting packet sniffing...")
    sniff(iface='Wi-Fi', prn=packet_handler, store=0, filter="ip")  # Change the interface and filter if necessary

if __name__ == "__main__":
    start_sniffing()


Starting packet sniffing...
Packet: Ether / IP / TCP 192.168.180.3:59767 > 23.76.157.96:http S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59803 > 23.76.157.113:http S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59858 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59859 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59861 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59862 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59863 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59865 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59866 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59867 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP 192.168.180.3:59870 > 192.168.180.248:domain S [1]| Malicious
Packet: Ether / IP / TCP